<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/03_Ciclo_Pedido_Magalu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import datetime
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
# Montar Drive
from google.colab import drive
drive.mount('/content/drive')
# Classe Dados
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content
from modulos_colab import dados
# Chaves e tokens de acesso
!cp /content/drive/MyDrive/Colab_Notebooks/Autenticacoes/chaves_tokens.py /content
%run chaves_tokens.py
# Querys de consulta
!cp /content/drive/MyDrive/Colab_Notebooks/querys/querys_cicloPedido_magalu.ipynb /content
%run querys_cicloPedido_magalu.ipynb

Base Parceiro

In [ ]:
def envio_parceiro():
    # Base pedidos envio parceiro
    base_pedidos_parceiro = dados.consulta_bigquery(projeto_gcp_1, query_pedidos_envio_parceiro).fillna(0)
    # Remover duplicados
    base_pedidos_parceiro_novo = base_pedidos_parceiro.drop_duplicates()
    base_pedidos_parceiro_novo['Classificação_Pedidos_Seller'] = 'Pedido Parceiro'
    # Selecionar pedidos sem entrega
    base1_pedidos_parceiro = base_pedidos_parceiro_novo.query('entrega_realizada=="N"')
    base1_pedidos_parceiro['Hoje'] = datetime.datetime.today().strftime('%Y-%m-%d')
    base1_pedidos_parceiro['Hoje'] = pd.to_datetime(base1_pedidos_parceiro['Hoje'], format='%Y-%m-%d')
    base1_pedidos_parceiro['data_prometida_entrega'] = pd.to_datetime(base1_pedidos_parceiro['data_prometida_entrega'])
    base1_pedidos_parceiro['Dias'] = (base1_pedidos_parceiro['data_prometida_entrega']-base1_pedidos_parceiro['Hoje'])
    base1_pedidos_parceiro['Dias'] = base1_pedidos_parceiro['Dias'].dt.days
    base1_pedidos_parceiro['data_aprovacao'] = pd.to_datetime(base1_pedidos_parceiro['data_aprovacao'], format='%Y-%m-%d',errors='coerce')
    # Status Pedido
    def status_pedido(base1_pedidos_parceiro):
        if base1_pedidos_parceiro['Dias'] < 0:
            return 'Atrasado'
        else: 
            return 'Pendente'
    base1_pedidos_parceiro['Classificação_Pedidos'] = base1_pedidos_parceiro.apply(status_pedido, axis=1)
    return base1_pedidos_parceiro

base1_pedidos_parceiro =  envio_parceiro()  


**Base MLE**

In [ ]:
def envio_mle():
    # Base pedidos mle
    base_pedidos_mle = dados.consulta_bigquery(projeto_gcp_1, query_pedidos_envio_mle).fillna(0)
    # Remover duplicados
    base_pedidos_mle_novo = base_pedidos_mle.drop_duplicates()
    base_pedidos_mle_novo['Classificação_Pedidos_Seller'] = 'Pedido Mle'
    # Selecionar pedidos sem entrega
    base1_pedidos_mle = base_pedidos_mle_novo.query('entrega_realizada=="N"')
    base1_pedidos_mle['Hoje'] = datetime.datetime.today().strftime('%Y-%m-%d')
    base1_pedidos_mle['Hoje'] = pd.to_datetime(base1_pedidos_mle['Hoje'], format='%Y-%m-%d')
    base1_pedidos_mle['data_limite_postagem'] = pd.to_datetime(base1_pedidos_mle['data_limite_postagem'], format='%Y-%m-%d')
    base1_pedidos_mle['Dias'] = (base1_pedidos_mle['data_limite_postagem']-base1_pedidos_mle['Hoje'])
    base1_pedidos_mle['Dias'] = base1_pedidos_mle['Dias'].dt.days
    base1_pedidos_mle['data_aprovacao'] = pd.to_datetime(base1_pedidos_mle['data_aprovacao'], format='%Y-%m-%d',errors='coerce')
    # Status Pedido
    def status_pedido(base1_pedidos_mle):
        if base1_pedidos_mle['Dias'] <= -1:
            return 'Atrasado'
        else: 
            return 'Pendente'
    # Classificação pedidos    
    base1_pedidos_mle['Classificação_Pedidos'] = base1_pedidos_mle.apply(status_pedido, axis=1)
    return base1_pedidos_mle


base1_pedidos_mle = envio_mle()


**Base Protocolos**

In [ ]:
# Protocolos
base_protocolos = dados.consulta_bigquery(projeto_gcp_1, query_protocolos).fillna('')
# Ajustes tabela protocolo
base_protocolos['order_id'] = base_protocolos['order_id'].str.extract('(\d+)').astype(object)



**Junção das Bases (Parceiro, MLE e Protocolos)**


In [ ]:
# Concatenação e ajustes para tabela resumo

resumo_pedidos_parceiro = pd.DataFrame(base1_pedidos_parceiro[['seller_id', 'score', 'nota_final','Assistido', 'Carteira', 'Mundo', 'Situacao_seller']])
resumo_pedidos_mle = pd.DataFrame(base1_pedidos_mle[['seller_id', 'score', 'nota_final','Assistido', 'Carteira', 'Mundo', 'Situacao_seller']])
sellers_pedidos = pd.concat([resumo_pedidos_parceiro, resumo_pedidos_mle]).drop_duplicates()

resumo_protocolos = pd.DataFrame(base_protocolos[['seller_id', 'score', 'nota_final','Assistido', 'Carteira', 'Mundo', 'Situacao_seller']])

seller_total = pd.concat([sellers_pedidos, resumo_protocolos]).drop_duplicates()

parceiro_atrasado = base1_pedidos_parceiro.query('Classificação_Pedidos_Seller=="Pedido Parceiro" & Classificação_Pedidos=="Atrasado"')
seller_pedidos_atrasados_parceiro = pd.DataFrame(pd.pivot_table(parceiro_atrasado, index='seller_id', columns='Classificação_Pedidos_Seller',  values='order_id', aggfunc='nunique')).fillna(0)

base_1 = pd.merge(seller_total, seller_pedidos_atrasados_parceiro, how='left', on='seller_id').fillna(0)
base_1.rename(columns={'Pedido Parceiro':'Pedido_Parceiro_atrasado'}, inplace=True)

mle_atrasado = base1_pedidos_mle.query('Classificação_Pedidos_Seller=="Pedido Mle" & Classificação_Pedidos=="Atrasado" & status_integra!="SHIPPED"')
seller_pedidos_atrasados_mle = pd.DataFrame(pd.pivot_table(mle_atrasado, index='seller_id', columns='Classificação_Pedidos_Seller',  values='order_id', aggfunc='nunique')).fillna(0)

base_2 = pd.merge(base_1, seller_pedidos_atrasados_mle, how='left', on='seller_id').fillna(0)
base_2.rename(columns={'Pedido Mle':'Pedido_Mle_atrasado'}, inplace=True)

parceiro_pendente = base1_pedidos_parceiro.query('Classificação_Pedidos_Seller=="Pedido Parceiro" & Classificação_Pedidos=="Pendente"')
seller_pedidos_pendente_parceiro = pd.DataFrame(pd.pivot_table(parceiro_pendente, index='seller_id', columns='Classificação_Pedidos_Seller',  values='order_id', aggfunc='nunique')).fillna(0)

base_3 = pd.merge(base_2, seller_pedidos_pendente_parceiro, how='left', on='seller_id').fillna(0)
base_3.rename(columns={'Pedido Parceiro':'Pedido_Parceiro_pendente'}, inplace=True)

mle_pendente = base1_pedidos_mle.query('Classificação_Pedidos_Seller=="Pedido Mle" & Classificação_Pedidos=="Pendente" & status_integra!="SHIPPED"')
seller_pedidos_pendente_mle_pendente = pd.DataFrame(pd.pivot_table(mle_pendente, index='seller_id', columns='Classificação_Pedidos_Seller',  values='order_id', aggfunc='nunique')).fillna(0)

base_4 = pd.merge(base_3, seller_pedidos_pendente_mle_pendente, how='left', on='seller_id').fillna(0)
base_4.rename(columns={'Pedido Mle':'Pedido_Mle_pendente'}, inplace=True)

protocolos = pd.pivot_table(base_protocolos, index='seller_id', values='protocolo', aggfunc='nunique').fillna(0)

base_5 = pd.merge(base_4, protocolos, how='left', on='seller_id').fillna(0)

base_aprovacao_parceiro = base1_pedidos_parceiro[['seller_id', 'data_aprovacao']]
base_aprovacao_mle = base1_pedidos_mle[['seller_id', 'data_aprovacao']]
base_aprovacao_tt = pd.concat([base_aprovacao_parceiro, base_aprovacao_mle])

qtd_aprovado = pd.DataFrame(pd.pivot_table(base_aprovacao_tt, index='seller_id',  values='data_aprovacao', aggfunc='count')).fillna(0)
qtd_aprovado.rename(columns={'data_aprovacao':'Qtd_aprovacao'}, inplace=True)

base_6 = pd.merge(base_5, qtd_aprovado, how='left', on='seller_id').fillna(0)

base_6['nota_final'] = round(pd.to_numeric(base_6['nota_final']),2)
base_6['nota_final'] = base_6['nota_final'].replace('.', ',')
base_6['Pedido_Parceiro_atrasado'] = base_6['Pedido_Parceiro_atrasado'].astype(int)
base_6['Pedido_Mle_atrasado'] = base_6['Pedido_Mle_atrasado'].astype(int)
#base_6['Pedido_Parceiro_pendente']=""  #Comentar após fechamento.
base_6['Pedido_Parceiro_pendente'] = base_6['Pedido_Parceiro_pendente'].astype(int)
#base_6['Pedido_Mle_pendente']=""    #Comentar após fechamento.
base_6['Pedido_Mle_pendente'] = base_6['Pedido_Mle_pendente'].astype(int)
base_6['protocolo'] = base_6['protocolo'].astype(int)
base_6['Qtd_aprovacao'] = base_6['Qtd_aprovacao'].astype(int)
base_6.drop_duplicates(inplace=True)


In [ ]:
# Divisão Sellers
divisao_sellers = pd.read_excel('/content/drive/MyDrive/Colab_Notebooks/arquivos/ciclo_pedido/magalu/Divisão Ciclo Magalu - Setembro22.xlsx')
base_7 = pd.merge(base_6, divisao_sellers, how='left', on='seller_id').fillna('sem_agente')

# Validação Sellers sem agente
def dist_sem_agentes(base_7):
    if base_7['Agentes'] != 'sem_agente':
        return base_7['Agentes']
    elif base_7['Agentes'] == 'sem_agente':
        return 'Rateio'
base_7['Agentes'] = base_7.apply(dist_sem_agentes, axis=1)

# Base para rateio
rateio = base_7.query('Agentes=="Rateio"')
base_com_agentes = base_7.query('Agentes!="Rateio"')

# lista de agentes
agentes = ['Bruna', 'Carmen', 'Leonardo', 'Rayane']
# Distribuição dos agentes
lista_agentes = []
x = 0
linhas = len(rateio['seller_id'])
while (x <= linhas):
    for item in agentes:
        lista_agentes.append(item)
        x+=1
# remover agentes 
remover = len(lista_agentes) - linhas

if len(lista_agentes) > linhas:
    del lista_agentes[0:remover]
else:
    lista_agentes

# Coluna  Agentes base rateio
rateio['Agentes'] = lista_agentes

# Concatenação bases
base_7 = pd.concat([base_com_agentes, rateio])


# Conferência rateio
base_agentes = base_7[['seller_id','Agentes']]
pd.pivot_table(base_7, index='Agentes', values='seller_id', aggfunc='count').head(7)


In [ ]:
# Imput na tabela resumo
base_9 = base_7.query('nota_final!=0.0')
base_9.to_gbq(destination_table='data.Tb_ciclo_pedido_magalu_resumo',project_id=projeto_gcp_2, if_exists='replace')


In [ ]:
# Tabela pedidos atrasados parceiro
atrasados_parceiro = base1_pedidos_parceiro.query('Classificação_Pedidos=="Atrasado" & Classificação_Pedidos_Seller=="Pedido Parceiro"')
#atrasados_parceiro['nota_final'] = atrasados_parceiro['nota_final']

atrasados_parceiro_agentes = pd.merge(atrasados_parceiro,base_agentes, how='left', on='seller_id' )

# Formatação pedidos
atrasados_parceiro_agentes['order_id'] = 'LU-' + atrasados_parceiro_agentes['order_id']
atrasados_parceiro_agentes['seller_mle'] = ""
atrasados_parceiro_agentes['tipo_entrega'] = 'Envio parceiro'

# Selecionar colunas para impute na tabela
atrasados_parceiro_v1 = atrasados_parceiro_agentes[['data_captado',
                                                    'order_id', 
                                                    'seller_id',
                                                    'status_maestro',
                                                    'data_prometida_entrega',
                                                    'entrega_realizada',
                                                    'data_entrega',
                                                    'entrega_no_prazo',
                                                    'seller_mle',
                                                    'tipo_entrega',
                                                    'Assistido',
                                                    'Carteira',
                                                    'Mundo',
                                                    'Situacao_seller',
                                                    'nota_final',
                                                    'score',
                                                    'status_integra',
                                                    'data_expedicao',
                                                    'data_aprovacao',
                                                    'data_limite_postagem',
                                                    'data_limite_postagem_mle',
                                                    'Hoje',
                                                    'Dias',
                                                    'Classificação_Pedidos',
                                                    'Classificação_Pedidos_Seller',
                                                    'Agentes']]

atrasados_parceiro_v1.rename(columns={'Classificação_Pedidos':'Classificacao_Pedidos','Classificação_Pedidos_Seller':'Classificacao_Pedidos_Seller'},inplace=True)
atrasados_parceiro_v1.to_gbq(destination_table='data.Tb_ciclo_pedido_atrasado_parceiro',project_id=projeto_gcp_2, if_exists='replace')



In [ ]:
# Tabela pedidos atrasados mle
atrasados_mle = base1_pedidos_mle.query('Classificação_Pedidos=="Atrasado" & Classificação_Pedidos_Seller=="Pedido Mle"')
#atrasados_mle['nota_final'] = round(atrasados_mle['nota_final'].astype(float),2)

atrasados_mle_agentes = pd.merge(atrasados_mle,base_agentes, how='left', on='seller_id' )

# Formatar pedidos
atrasados_mle_agentes['order_id'] = 'LU-' + atrasados_mle_agentes['order_id']
atrasados_mle_agentes['seller_mle'] = atrasados_mle_agentes['seller_id']
atrasados_mle_agentes['tipo_entrega'] = 'Envio MLE'

# Selecionar colunas para impute na tabela
atrasados_mle_v1 = atrasados_mle_agentes[['data_captado',
                                            'order_id', 
                                            'seller_id',
                                            'status_maestro',
                                            'data_prometida_entrega',
                                            'entrega_realizada',
                                            'data_entrega',
                                            'entrega_no_prazo',
                                            'seller_mle',
                                            'opcao_entrega',
                                            'Assistido',
                                            'Carteira',
                                            'Mundo',
                                            'Situacao_seller',
                                            'nota_final',
                                            'score',
                                            'status_integra',
                                            'data_expedicao',
                                            'data_aprovacao',
                                            'data_limite_postagem',
                                            'data_limite_postagem_mle',
                                            'Hoje',
                                            'Dias',
                                            'Classificação_Pedidos',
                                            'Classificação_Pedidos_Seller',
                                            'Agentes']]

atrasados_mle_v1.rename(columns={'Classificação_Pedidos':'Classificacao_Pedidos','Classificação_Pedidos_Seller':'Classificacao_Pedidos_Seller'},inplace=True)
atrasados_mle_v1.to_gbq(destination_table='data.Tb_ciclo_pedido_atrasado_mle',project_id=projeto_gcp_2, if_exists='replace')


In [ ]:
  # Tabela pedidos pendentes envio parceiro
base1_pedidos_parceiro['tipo_entrega'] = 'Envio parceiro'
pendentes_envio_parceiro = base1_pedidos_parceiro.query('Classificação_Pedidos=="Pendente" & tipo_entrega=="Envio parceiro"')


#pendentes_envio_parceiro['nota_final'] = round(pendentes_envio_parceiro['nota_final'].astype(float),2)
pendentes_envio_parceiro['order_id'] = 'LU-' + pendentes_envio_parceiro['order_id']

# Imput tabela pendentes envio parceiro

pendentes_envio_parceiro.rename(columns={'Classificação_Pedidos':'Classificacao_Pedidos','Classificação_Pedidos_Seller':'Classificacao_Pedidos_Seller'},inplace=True)
pendentes_envio_parceiro.to_gbq(destination_table='data.Tb_ciclo_pedido_pendentes_parceiro',project_id=projeto_gcp_2, if_exists='replace')



In [ ]:
# Tabela pedidos pendentes envio mle
base1_pedidos_mle['tipo_entrega'] = 'Envio MLE'
pendentes_envio_mle = base1_pedidos_mle.query('Classificação_Pedidos=="Pendente" & status_integra!="SHIPPED" & tipo_entrega=="Envio MLE"')

#pendentes_envio_mle['nota_final'] = round(pendentes_envio_mle['nota_final'].astype(float),2)
pendentes_envio_mle['order_id'] = 'LU-' + pendentes_envio_mle['order_id']

# Imput pedidos pendentes envio mle

pendentes_envio_mle.rename(columns={'Classificação_Pedidos':'Classificacao_Pedidos','Classificação_Pedidos_Seller':'Classificacao_Pedidos_Seller'},inplace=True)
pendentes_envio_mle.to_gbq(destination_table='data.Tb_ciclo_pedido_pendentes_mle',project_id=projeto_gcp_2, if_exists='replace')



In [ ]:
# Tabela protocolos
'''base_protocolos['nota_final'] = round(base_protocolos['nota_final'].astype(float),2)'''
#base_protocolos['nota_final'] = round(pd.to_numeric(base_protocolos['nota_final']),2)
base_protocolos['order_id'] = 'LU-' + base_protocolos['order_id']

# Imput pedidos pendentes envio mle

base_protocolos.to_gbq(destination_table='data.Tb_protocolos_magalu',project_id=projeto_gcp_2, if_exists='replace')


In [ ]:
# Base Fulfillment
base_fulfillmente = dados.consulta_bigquery(projeto_gcp_1, query_fulfillment)
base_fulfillmente.to_gbq(destination_table='data.Tb_pedidos_fulfillment',project_id=projeto_gcp_2, if_exists='replace')



In [ ]:
print("Ciclo pedido magalu atualizado!")


In [ ]:
'''
# Pedidos processados - Consulta BigQuery
# projeto
maga_bigdata = 'maga-bigdata'
#autenticação e select
client = bigquery.Client(project=maga_bigdata)

sql= """SELECT  
                  so.seller_id, cast(DATETIME_ADD(CAST(processado.date AS DATETIME), INTERVAL -3 HOUR)as date) as data_pedido_processado,
                  count(distinct so.uuid)  as pedido_processado -- , sum(cast(so.amount as FLOAT64)) as valor_pedido
                  FROM maga-bigdata.maestro.order as o
                  left join maga-bigdata.maestro.sub_order as so on so.order_uuid = o.uuid
                  left join maga-bigdata.maestro.sales_channel as sc on o.sales_channel_id = sc.id
                  inner join (SELECT distinct status_id, min(date) as date, sub_order_uuid 
                           FROM maga-bigdata.maestro.event 
                           where duplicated = false
                           group by status_id,sub_order_uuid) as processado 
                         on so.uuid = processado.sub_order_uuid and processado.status_id =2
                  where so.seller_id <> 'magazineluiza'  
                  and sc.organization_uuid = '4da25f48-4193-45de-b4ed-9b8c93b7f987'
                  and cast(DATETIME_ADD(CAST(processado.date AS DATETIME), INTERVAL -3 HOUR)as date) between '2021-01-01' and '2022-04-30'
                  group by so.seller_id,  cast(DATETIME_ADD(CAST(processado.date AS DATETIME), INTERVAL -3 HOUR)as date)"""

# Select salvo na variavel
pedidos_processados_2021 = client.query(sql).to_dataframe().fillna(0)

pedidos_processados_2021['data_pedido_processado'] = pd.to_datetime(pedidos_processados_2021['data_pedido_processado'])
pedidos_processados_2021['mes'] = pedidos_processados_2021['data_pedido_processado'].dt.month
pedidos_processados_2021['ano'] = pedidos_processados_2021['data_pedido_processado'].dt.year
dinamica = pd.pivot_table(pedidos_processados_2021, index='seller_id', columns=['ano','mes'], values='pedido_processado', aggfunc='sum').fillna(0)
#dinamica.to_csv('/content/drive/MyDrive/dados_pedidos/Magalu/Ciclo_pedido/pedidos_processados_2021.csv')
dinamica.head()
'''